In [27]:
%%bash

bucketname="$(echo $USER"-sql-workshop"|cut -d "-" -f2-)"
echo $bucketname

cp /etc/skel/.env ~/
echo "BUCKETNAME="$bucketname >> ~/.env

aws s3api create-bucket --bucket $bucketname --region us-east-1
aws s3api list-buckets | grep $bucketname 

arhodes-sql-workshop
/arhodes-sql-workshop
BUCKETS	2021-08-30T19:29:32.000Z	arhodes-sql-workshop


In [28]:
from dotenv import load_dotenv
load_dotenv()

True

In [29]:
import os 
os.getenv('BUCKETNAME')

'arhodes-sql-workshop'

In [30]:
import os
os.environ['ATHENA_USER'] = os.getenv('AWS_ACCESS_KEY_ID')
os.environ['ATHENA_PASSWORD'] = os.getenv('AWS_SECRET_ACCESS_KEY')
os.environ['ATHENA_REGION']= 'us-east-1'
os.environ['ATHENA_STAGING']= os.getenv('BUCKETNAME')
os.environ['ATHENA_DATABASE']='sra'

In [31]:
import pyathena
import pandas as pd


#This line of code gives the connection information to AWS, this variable is used in the sql commands

athena_conn = pyathena.connect(aws_access_key_id=os.environ['ATHENA_USER'], ##credentials of aws_access_key_id
                 aws_secret_access_key=os.environ['ATHENA_PASSWORD'], ##credentials of aws_secret_access_key
                 s3_staging_dir=os.environ['ATHENA_STAGING'], ##where the athena query result saved - checked in S3 ,
                 region_name=os.environ['ATHENA_REGION']) ##the region you set for Athena




In [32]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [43]:
import sqlalchemy
from urllib.parse import quote_plus
from urllib.parse import quote
from sqlalchemy.engine import create_engine


AWS_ACCESS_KEY = os.environ['ATHENA_USER']
AWS_SECRET_KEY = os.environ['ATHENA_PASSWORD']
SCHEMA_NAME = os.environ['ATHENA_DATABASE']
S3_STAGING_DIR = "s3://" + os.environ['ATHENA_STAGING']
AWS_REGION = "us-east-1"


conn_str = (
    "awsathena+rest://{aws_access_key_id}:{aws_secret_access_key}@"
    "athena.{region_name}.amazonaws.com/"
    "{schema_name}?s3_staging_dir{s3_staging_dir}"
)



# Create the SQLAlchemy connection. Note that you need to have pyathena installed for this.
connection = conn_str.format(
        aws_access_key_id=quote_plus(AWS_ACCESS_KEY),
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION,
        schema_name=SCHEMA_NAME,
        s3_staging_dir=S3_STAGING_DIR,
    )

engine = create_engine(connection)

#&work_group=primary"

In [45]:
%config SqlMagic.displaycon = False

In [46]:
%%sql 

SELECT acc, librarysource, bioproject, geo_loc_name_country_calc
FROM sra.metadata
WHERE organism = 'Homo sapiens'
LIMIT 5

Done.


acc,librarysource,bioproject,geo_loc_name_country_calc
ERR2864488,GENOMIC,PRJEB29458,None
ERR2777656,GENOMIC,PRJEB28471,None
ERR2809354,GENOMIC,PRJEB28664,None
ERR2809215,GENOMIC,PRJEB28664,None
ERR2809488,GENOMIC,PRJEB28664,None


In [ ]:
#Use this code block by removing the "#" in front of the following commands 
#After the workshop to set up credentials for your AWS Account To use this Notebook
#This has already been done on the workshop server

#%env AWS_ACCESS_KEY_ID=<your_AWS_Access ID>
#%env AWS_SECRET_ACCESS_KEY=<your AWS Secret>
#%env BUCKETNAME=<your staging bucket name>


#!aws configure set aws_access_key_id "$AWS_ACCESS_KEY_ID"
#!aws configure set aws_secret_access_key "$AWS_SECRET_ACCESS_KEY" 
#!aws configure set region "us-east-1" 
#!aws configure set output "text"


#!sudo pip install pyathena 
#!sudo pip install ipython-sql
#!pip install sqlalchemy

